In [1]:
from sympy import *
import numpy as np
import cirq
from sympy.physics import quantum
from sympy import abc

In [2]:
sigma_i = Matrix([[1, 0], [0, 1]])
sigma_x = Matrix([[0, 1], [1, 0]])
sigma_y = Matrix([[0, -I], [I, 0]])
sigma_z = Matrix([[1, 0], [0, -1]])

zero_ket = Matrix([[1], [0]])
one_ket = Matrix([[0], [1]])
zero_zero = zero_ket * quantum.Dagger(zero_ket)
one_one = one_ket * quantum.Dagger(one_ket)
zero_one = zero_ket * quantum.Dagger(one_ket)
one_zero = one_ket * quantum.Dagger(zero_ket)


In [3]:
def flatten_by_row(mat):
    d1, d2 = mat.shape
    return mat.reshape(d1 * d2, 1)

def flatten_by_col(mat):
    d2 = mat.shape[1]
    vec = mat[:,0]
    for i in range(1, d2):
        vec = vec.col_join(mat[:,i])
    return vec
    

### Example: Amplitude damping channel


In [49]:
E0 = zero_zero + sqrt(1 - abc.epsilon) * one_one
E1 = sqrt(abc.epsilon) * zero_one
d = E0.shape[0]
E0_vec = flatten_by_row(E0)
E1_vec = flatten_by_row(E1)

In [50]:
kronecker_product(E0, E0)

Matrix([
[1,                 0,                 0,           0],
[0, sqrt(1 - epsilon),                 0,           0],
[0,                 0, sqrt(1 - epsilon),           0],
[0,                 0,                 0, 1 - epsilon]])

In [51]:
kronecker_product(E1, E1)

Matrix([
[0, 0, 0, epsilon],
[0, 0, 0,       0],
[0, 0, 0,       0],
[0, 0, 0,       0]])

In [5]:
K = kronecker_product(E0, E0) + kronecker_product(E1, E1)  # superoperator
K

Matrix([
[1,                 0,                 0,     epsilon],
[0, sqrt(1 - epsilon),                 0,           0],
[0,                 0, sqrt(1 - epsilon),           0],
[0,                 0,                 0, 1 - epsilon]])

In [6]:
K.inv()

Matrix([
[1,                   0,                   0, -epsilon/(1 - epsilon)],
[0, 1/sqrt(1 - epsilon),                   0,                      0],
[0,                   0, 1/sqrt(1 - epsilon),                      0],
[0,                   0,                   0,        1/(1 - epsilon)]])

by observation, the inverse components of $K$ is
$$
F_0 = \begin{bmatrix} 1 & 0 \\ 0 & \frac{1}{\sqrt{1-\epsilon}} \end{bmatrix}, \quad F_1 = \begin{bmatrix} 0 & \sqrt{\frac{\epsilon}{1 - \epsilon}} \\ 0 & 0 \end{bmatrix}
$$

and $\mathcal{E}^{-1}(\cdot) = F_0(\cdot)F_0^\dagger - F_1(\cdot)F_1^\dagger$

In [7]:
F0 = zero_zero + 1 / sqrt(1 - abc.epsilon) * one_one
F1 = sqrt(abc.epsilon / (1 - abc.epsilon)) * zero_one

In [8]:
kronecker_product(F0, F0) - kronecker_product(F1, F1)

Matrix([
[1,                   0,                   0, -epsilon/(1 - epsilon)],
[0, 1/sqrt(1 - epsilon),                   0,                      0],
[0,                   0, 1/sqrt(1 - epsilon),                      0],
[0,                   0,                   0,        1/(1 - epsilon)]])

Then let us see the Choi matrix representation and its inverse's Choi matrix

In [9]:
E0_vec = flatten_by_row(E0)
E1_vec = flatten_by_row(E1)
F0_vec = flatten_by_row(F0)
F1_vec = flatten_by_row(F1)

In [10]:
J = E0_vec * E0_vec.T + E1_vec * E1_vec.T  # Choi matrix
J

Matrix([
[                1,       0, 0, sqrt(1 - epsilon)],
[                0, epsilon, 0,                 0],
[                0,       0, 0,                 0],
[sqrt(1 - epsilon),       0, 0,       1 - epsilon]])

In [11]:
J_inv = F0_vec * F0_vec.T - F1_vec * F1_vec.T
J_inv

Matrix([
[                  1,                      0, 0, 1/sqrt(1 - epsilon)],
[                  0, -epsilon/(1 - epsilon), 0,                   0],
[                  0,                      0, 0,                   0],
[1/sqrt(1 - epsilon),                      0, 0,     1/(1 - epsilon)]])

In [12]:
F0_vec * F0_vec.T

Matrix([
[                  1, 0, 0, 1/sqrt(1 - epsilon)],
[                  0, 0, 0,                   0],
[                  0, 0, 0,                   0],
[1/sqrt(1 - epsilon), 0, 0,     1/(1 - epsilon)]])

In [13]:
F1_vec * F1_vec.T

Matrix([
[0,                     0, 0, 0],
[0, epsilon/(1 - epsilon), 0, 0],
[0,                     0, 0, 0],
[0,                     0, 0, 0]])

In [14]:
J.is_positive_definite

False

In [15]:
# J_1 = \left(\begin{matrix}1 & 0 & 0 & \sqrt{1 - \epsilon}\\0 & 0 & 0 & 0\\0 & 0 & 0 & 0\\ \sqrt{1 - \epsilon} & 0 & 0 & 1\end{matrix}\right),\, \eta_1 = \frac{1}{1-\epsilon}
J1 = Matrix([[1, 0, 0, sqrt(1 - abc.epsilon)], [0, 0, 0, 0], [0, 0, 0, 0], [sqrt(1 - abc.epsilon), 0, 0, 1]])
eta1 = 1 / (1 - abc.epsilon)
J1

Matrix([
[                1, 0, 0, sqrt(1 - epsilon)],
[                0, 0, 0,                 0],
[                0, 0, 0,                 0],
[sqrt(1 - epsilon), 0, 0,                 1]])

In [16]:
# J_2 = \left(\begin{matrix} 1 & 0 & 0 & 0\\0 & 1 & 0 & 0\\0 & 0 & 0 & 0\\0 & 0 & 0 & 0\end{matrix}\right),\,\eta_2 = \frac{\epsilon}{1-\epsilon}
J2 = Matrix([[1, 0, 0, 0], [0, 1, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0]])
eta2 = abc.epsilon / (1 - abc.epsilon)
J2

Matrix([
[1, 0, 0, 0],
[0, 1, 0, 0],
[0, 0, 0, 0],
[0, 0, 0, 0]])

$$J = \eta_1 J_1 - \eta_2 J_2$$

### Example: Depolarizing channel


In [52]:
E0 = sqrt(1 - abc.epsilon) * sigma_i
E1 = sqrt(abc.epsilon / 3) * sigma_x
E2 = sqrt(abc.epsilon / 3) * sigma_y
E3 = sqrt(abc.epsilon / 3) * sigma_z
E0_conj, E1_conj, E2_conj, E3_conj = E0, E1, sqrt(abc.epsilon / 3) * sigma_y.conjugate(), sqrt(abc.epsilon / 3) * sigma_z.conjugate()
E0_vec = flatten_by_row(E0)
E1_vec = flatten_by_row(E1)
E2_vec = flatten_by_row(E2)
E3_vec = flatten_by_row(E3)

In [53]:
K = kronecker_product(E0, E0_conj) + kronecker_product(E1, E1_conj) + kronecker_product(E2, E2_conj) + kronecker_product(E3, E3_conj)

In [56]:
K

Matrix([
[1 - 2*epsilon/3,               0,               0,     2*epsilon/3],
[              0, 1 - 4*epsilon/3,               0,               0],
[              0,               0, 1 - 4*epsilon/3,               0],
[    2*epsilon/3,               0,               0, 1 - 2*epsilon/3]])

In [58]:
K.inv()

Matrix([
[(2*epsilon - 3)/(4*epsilon - 3),                   0,                   0,       2*epsilon/(4*epsilon - 3)],
[                              0, 1/(1 - 4*epsilon/3),                   0,                               0],
[                              0,                   0, 1/(1 - 4*epsilon/3),                               0],
[     -2*epsilon/(3 - 4*epsilon),                   0,                   0, (2*epsilon - 3)/(4*epsilon - 3)]])

How to get the the `K.inv()`?

In [59]:
kronecker_product(E0, E0_conj)

Matrix([
[1 - epsilon,           0,           0,           0],
[          0, 1 - epsilon,           0,           0],
[          0,           0, 1 - epsilon,           0],
[          0,           0,           0, 1 - epsilon]])

In [60]:
kronecker_product(E1, E1_conj)

Matrix([
[        0,         0,         0, epsilon/3],
[        0,         0, epsilon/3,         0],
[        0, epsilon/3,         0,         0],
[epsilon/3,         0,         0,         0]])

In [61]:
kronecker_product(E2, E2_conj)

Matrix([
[        0,          0,          0, epsilon/3],
[        0,          0, -epsilon/3,         0],
[        0, -epsilon/3,          0,         0],
[epsilon/3,          0,          0,         0]])

In [63]:
kronecker_product(E3, E3_conj)

Matrix([
[epsilon/3,          0,          0,         0],
[        0, -epsilon/3,          0,         0],
[        0,          0, -epsilon/3,         0],
[        0,          0,          0, epsilon/3]])

By leveraging components of `K.inv()` ...

In [81]:
component0 = (abc.epsilon - 3) / (1 - abc.epsilon) / (4 * abc.epsilon - 3) * kronecker_product(E0, E0_conj)
component0

Matrix([
[(epsilon - 3)/(4*epsilon - 3),                             0,                             0,                             0],
[                            0, (epsilon - 3)/(4*epsilon - 3),                             0,                             0],
[                            0,                             0, (epsilon - 3)/(4*epsilon - 3),                             0],
[                            0,                             0,                             0, (epsilon - 3)/(4*epsilon - 3)]])

In [89]:
kronecker_product(E0, E0_conj)

Matrix([
[1 - epsilon,           0,           0,           0],
[          0, 1 - epsilon,           0,           0],
[          0,           0, 1 - epsilon,           0],
[          0,           0,           0, 1 - epsilon]])

In [75]:
component1 = 3 / (4 * abc.epsilon - 3) * (kronecker_product(E1, E1_conj) + kronecker_product(E2, E2_conj) + kronecker_product(E3, E3_conj))
component1


Matrix([
[  epsilon/(4*epsilon - 3),                        0,                        0, 2*epsilon/(4*epsilon - 3)],
[                        0, -epsilon/(4*epsilon - 3),                        0,                         0],
[                        0,                        0, -epsilon/(4*epsilon - 3),                         0],
[2*epsilon/(4*epsilon - 3),                        0,                        0,   epsilon/(4*epsilon - 3)]])

In [84]:
simplify(component0 + component1) == simplify(K.inv())

True

Thus, we can get the inverse of the Choi matrix of the depolarizing channel.

$$
F_0 = \sqrt{1 + \epsilon_\textrm{inv}}I, \quad F_1 = \sqrt{\frac{\epsilon_{\textrm{inv}}}{3}} X, \quad F_2 = \sqrt{\frac{\epsilon_{\textrm{inv}}}{3}} Y, \quad F_3 = \sqrt{\frac{\epsilon_{\textrm{inv}}}{3}} Z
$$

$
\sqrt{\epsilon_{\textrm{inv}}} = \sqrt{\frac{3\epsilon}{3 - 4\epsilon}}
$, such that $\epsilon_{\textrm{inv}} \geq 0$ usually holds.


and $\mathcal{E}^{-1}(\cdot) = F_0(\cdot)F_0^\dagger - F_1(\cdot)F_1^\dagger - F_2(\cdot)F_2^\dagger - F_3(\cdot)F_3^\dagger$

In [121]:
eps_inv = 3 * abc.epsilon / (3 - 4 * abc.epsilon)
F0 = sqrt(1 + eps_inv) * sigma_i
F1 = sqrt(eps_inv / 3) * sigma_x
F2 = sqrt(eps_inv / 3) * sigma_y
F3 = sqrt(eps_inv / 3) * sigma_z
F0_conj, F1_conj, F2_conj, F3_conj = F0, F1, sqrt(eps_inv / 3) * sigma_y.conjugate(), sqrt(eps_inv / 3) * sigma_z.conjugate()

In [122]:
simplify(kronecker_product(F0, F0_conj)) == component0

True

In [127]:
simplify(kronecker_product(F1, F1_conj) + kronecker_product(F2, F2_conj) + kronecker_product(F3, F3_conj)) == - component1

True

In [134]:
K_inv = kronecker_product(F0, F0_conj) - kronecker_product(F1, F1_conj) - kronecker_product(F2, F2_conj) - kronecker_product(F3, F3_conj)
K_inv = simplify(K_inv)
K_inv == simplify(K.inv())

True

### Example: 2-qubit global depolarizing channel


In [140]:
paulis = (sigma_i, sigma_x, sigma_y, sigma_z)
paulis_2q = (
    (kronecker_product(sigma_i, sigma_i), kronecker_product(sigma_i, sigma_x), kronecker_product(sigma_i, sigma_y), kronecker_product(sigma_i, sigma_z)),
    (kronecker_product(sigma_x, sigma_i), kronecker_product(sigma_x, sigma_x), kronecker_product(sigma_x, sigma_y), kronecker_product(sigma_x, sigma_z)),
    (kronecker_product(sigma_y, sigma_i), kronecker_product(sigma_y, sigma_x), kronecker_product(sigma_y, sigma_y), kronecker_product(sigma_y, sigma_z)),
    (kronecker_product(sigma_z, sigma_i), kronecker_product(sigma_z, sigma_x), kronecker_product(sigma_z, sigma_y), kronecker_product(sigma_z, sigma_z))
)


In [145]:
kraus_ops = []
for i in range(4):
    for j in range(4):
        if i == 0 and j == 0:
            kraus_ops.append(sqrt(1 - abc.epsilon) * paulis_2q[i][j])
        else:
            kraus_ops.append(sqrt(abc.epsilon / 15) * paulis_2q[i][j])
kraus_ops_conj = []
for i in range(4):
    for j in range(4):
        if i == 0 and j == 0:
            kraus_ops_conj.append(sqrt(1 - abc.epsilon) * paulis_2q[i][j].conjugate())
        else:
            kraus_ops_conj.append(sqrt(abc.epsilon / 15) * paulis_2q[i][j].conjugate())

In [165]:
res = zeros(4, 4)
for op in kraus_ops:
    res = res + op * op
assert res == eye(4), "does not satisfy the completeness relation \sum_i K_i^\dagger K_i = I"

In [166]:
# super-operator
K = zeros(16, 16)
for op, op_conj in zip(kraus_ops, kraus_ops_conj):
    K = K + kronecker_product(op, op_conj)

In [168]:
K.inv()

Matrix([
[(4*epsilon - 15)/(16*epsilon - 15),                     0,                     0,                     0,                     0,        4*epsilon/(16*epsilon - 15),                     0,                     0,                     0,                     0,        4*epsilon/(16*epsilon - 15),                     0,                     0,                     0,                     0,        4*epsilon/(16*epsilon - 15)],
[                                 0, 1/(1 - 16*epsilon/15),                     0,                     0,                     0,                                  0,                     0,                     0,                     0,                     0,                                  0,                     0,                     0,                     0,                     0,                                  0],
[                                 0,                     0, 1/(1 - 16*epsilon/15),                     0,                     0,                   

In [169]:
eps_inv = 15 * abc.epsilon / (15 - 16 * abc.epsilon)
kraus_inv_ops = []
for i in range(4):
    for j in range(4):
        if i == 0 and j == 0:
            kraus_inv_ops.append(sqrt(1 + eps_inv) * paulis_2q[i][j])
        else:
            kraus_inv_ops.append(sqrt(eps_inv / 15) * paulis_2q[i][j])
kraus_inv_ops_conj = []
for i in range(4):
    for j in range(4):
        if i == 0 and j == 0:
            kraus_inv_ops_conj.append(sqrt(1 + eps_inv) * paulis_2q[i][j].conjugate())
        else:
            kraus_inv_ops_conj.append(sqrt(eps_inv / 15) * paulis_2q[i][j].conjugate())

In [172]:
K_inv = zeros(16, 16)
for i, (op, op_conj) in enumerate(zip(kraus_inv_ops, kraus_inv_ops_conj)):
    if i == 0:
        K_inv = K_inv + kronecker_product(op, op_conj)
    else:
        K_inv = K_inv - kronecker_product(op, op_conj)
K_inv = simplify(K_inv)

In [174]:
K_inv == simplify(K.inv())

True